In [155]:
## Compute average quarterly return and holding periods of each investor

import pandas as pd
df_SF3 = pd.read_csv( './data/SHARADAR_SF3_ce320d02f19d0b5d04c9557e0bc16680.zip') # read SF3 table
df_SF3 = df_SF3[df_SF3['securitytype'] == 'SHR']

In [156]:
#Read list of investors who invested less than 1,000 shares from 2013 to 2019.
df_investors = pd.read_csv('./data/Investor.csv', header=None, sep = '\t')
df_investors = df_investors.rename(columns = {0:'investorID', 1:'investorname'})

SF3_dates = pd.to_datetime(df_SF3.calendardate.unique()) # collect dates of investor filings
SF3_dates_rev = SF3_dates.values[::-1]

for t in range(len(SF3_dates_rev)-1):    
    print('t=',t)
    #Holdings in quater t and t+1 respectively
    prevHoldings = df_SF3.query('calendardate == "{}"'.format(str(SF3_dates_rev[t])[:10]))[['investorname', 'ticker', 'units', 'price']]
    prevHoldings = pd.merge(prevHoldings, df_investors)[['investorID', 'ticker', 'units', 'price']]
    prevHoldings = prevHoldings[prevHoldings['price'] > 0]
    currHoldings = df_SF3.query('calendardate == "{}"'.format(str(SF3_dates_rev[t+1])[:10]))[['investorname', 'ticker', 'units', 'price']]
    currHoldings = pd.merge(currHoldings, df_investors)[['investorID', 'ticker', 'units', 'price']]
    currHoldings = currHoldings[currHoldings['price'] > 0]
    
    #Holdings in quater t+1 but not in t
    newHoldings = currHoldings.merge(prevHoldings, on=['investorID', 'ticker'], how='left')
    newHoldings = newHoldings[newHoldings.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x']]
    newHoldings = newHoldings.rename(columns = {'units_x':'units', 'price_x':'price'})
    newHoldings['invest_period'] = t+1
    
    #Holdings in quater t but not in t+1
    pastHoldings = prevHoldings.merge(currHoldings, on=['investorID', 'ticker'], how='left')
    pastHoldings = pastHoldings[pastHoldings.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x']]
    pastHoldings = pastHoldings.rename(columns = {'units_x':'units', 'price_x':'price'})
    pastHoldings['divest_period'] = t+1
    
    #Holdings both in quater t and t+1
    diffHoldings = currHoldings.merge(prevHoldings, on=['investorID', 'ticker'], how='left')
    diffHoldings = diffHoldings[diffHoldings.units_y.notnull()][['investorID','ticker', 'units_x', 'price_x', 'units_y', 'price_y']]
    
    #Extract tickers which did consolidation/splitting shares (or doughtful data which changed 10+ times) in quater t+1
    scaledHoldings = diffHoldings[((diffHoldings['units_y']/diffHoldings['units_x']>4)&(diffHoldings['price_x']/diffHoldings['price_y']>4))|((diffHoldings['units_x']/diffHoldings['units_y']>4)&(diffHoldings['price_y']/diffHoldings['price_x']>4))|((diffHoldings['price_x']/diffHoldings['price_y']>10)|(diffHoldings['price_x']/diffHoldings['price_y']>10))]
    diffHoldings = diffHoldings[~(((diffHoldings['units_y']/diffHoldings['units_x']>4)&(diffHoldings['price_x']/diffHoldings['price_y']>4))|((diffHoldings['units_x']/diffHoldings['units_y']>4)&(diffHoldings['price_y']/diffHoldings['price_x']>4))|((diffHoldings['price_x']/diffHoldings['price_y']>10)|(diffHoldings['price_x']/diffHoldings['price_y']>10)))]
    diffHoldings['units'] = diffHoldings['units_x'] - diffHoldings['units_y']
    diffHoldings['price'] = (diffHoldings['price_x'] + diffHoldings['price_y']) / 2

    beforeScaledHoldings = scaledHoldings[['investorID', 'ticker', 'units_y', 'price_y']]
    beforeScaledHoldings = beforeScaledHoldings.rename(columns = {'units_y':'units', 'price_y':'price'})
    beforeScaledHoldings['divest_period'] = t+1
    pastHoldings = pd.concat([pastHoldings, beforeScaledHoldings])
    
    afterScaledHoldings = scaledHoldings[['investorID', 'ticker', 'units_x', 'price_x']]
    afterScaledHoldings = afterScaledHoldings.rename(columns = {'units_x':'units', 'price_x':'price'})
    afterScaledHoldings['invest_period'] = t+1
    newHoldings = pd.concat([newHoldings, afterScaledHoldings])    
    
    #culculate return of wholly divested holdings
    if t > 0:
        pastHoldings = pastHoldings.merge(accmDivests, on=['investorID', 'ticker'], how='left')
        pastHoldings.fillna(0, inplace = True) 
        pastHoldings['units'] = pastHoldings['units_x'] + pastHoldings['units_y'] 
        pastHoldings['price'] = (pastHoldings['units_x']*pastHoldings['price_x'] + pastHoldings['units_y']*pastHoldings['price_y']) / pastHoldings['units']
        pastHoldings = pastHoldings[['investorID','ticker', 'units', 'price', 'divest_period']]
        accmInvests = accmInvests.merge(pastHoldings, on=['investorID', 'ticker'], how='left')
        if t == 1:
            investReturn = accmInvests[accmInvests.units_y.notnull()]
        else:
            investReturn = pd.concat([investReturn, accmInvests[accmInvests.units_y.notnull()]])

        #eliminate wholly divested holdings from accmInvests
        accmInvests = accmInvests[accmInvests.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x', 'invest_period']]
        accmInvests = accmInvests.rename(columns = {'units_x':'units', 'price_x':'price'})

        #eliminate wholly divested holdings from accmDivests
        accmDivests = accmDivests.merge(pastHoldings, on=['investorID', 'ticker'], how='left')
        accmDivests = accmDivests[accmDivests.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x']]
        accmDivests = accmDivests.rename(columns = {'units_x':'units', 'price_x':'price'})        

    #Increased holdings from quater t to t+1
    addHoldings = diffHoldings[diffHoldings['units'] > 0][['investorID','ticker', 'units', 'price']]
    
    #Decreased holdings from quater t to t+1
    divests = diffHoldings[diffHoldings['units'] < 0][['investorID','ticker', 'units', 'price']]
    divests['units'] = - divests['units']
        
    #Accumlate investments
    if t == 0:
        #Accumlate investment (Consider that all the holdings in quater t=0 invested at t=0)
        prevHoldings['invest_period'] = t        
        accmInvests = prevHoldings
    accmInvests = pd.concat([accmInvests, newHoldings])
    accmInvests = accmInvests.merge(addHoldings, on=['investorID', 'ticker'], how='left')
    accmInvests.fillna(0, inplace = True) 
    accmInvests['units'] = accmInvests['units_x'] + accmInvests['units_y'] 
    accmInvests['price'] = (accmInvests['units_x']*accmInvests['price_x'] + accmInvests['units_y']*accmInvests['price_y']) / accmInvests['units']
    accmInvests = accmInvests[['investorID','ticker', 'units', 'price', 'invest_period']]
        
    #Accumlate partial divestments
    if t == 0:
        accmDivests = divests
    if t > 0:
        #new partial divestments
        newDivests = divests.merge(accmDivests, on=['investorID', 'ticker'], how='left')
        newDivests = newDivests[newDivests.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x']]
        newDivests = newDivests.rename(columns = {'units_x':'units', 'price_x':'price'})
        
        #additional partially sold holdings
        addDivests = divests.merge(newDivests,  on=['investorID', 'ticker'], how='left')
        addDivests = addDivests[addDivests.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x']]
        addDivests = addDivests.rename(columns = {'units_x':'units', 'price_x':'price'})
        
        #update accumlated partially sold holdings
        accmDivests = accmDivests.merge(addDivests, on=['investorID', 'ticker'], how='left')
        accmDivests.fillna(0, inplace = True) 
        accmDivests['units'] = accmDivests['units_x'] + accmDivests['units_y'] 
        accmDivests['price'] = (accmDivests['units_x']*accmDivests['price_x'] + accmDivests['units_y']*accmDivests['price_y']) / accmDivests['units']
        accmDivests = accmDivests[['investorID','ticker', 'units', 'price']]
        accmDivests = pd.concat([accmDivests, newDivests])            
        
    #Consider all the holdings reft at the last quater divested at the quater
    if t == len(SF3_dates_rev)-2:
        newDivests = currHoldings.merge(accmDivests, on=['investorID', 'ticker'], how='left')
        newDivests = newDivests[newDivests.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x']]
        newDivests = newDivests.rename(columns = {'units_x':'units', 'price_x':'price'})
        addDivests = currHoldings.merge(newDivests,  on=['investorID', 'ticker'], how='left')
        addDivests = addDivests[addDivests.units_y.isnull()][['investorID','ticker', 'units_x', 'price_x']]
        addDivests = addDivests.rename(columns = {'units_x':'units', 'price_x':'price'})
        accmDivests = accmDivests.merge(addDivests, on=['investorID', 'ticker'], how='left')
        accmDivests.fillna(0, inplace = True) 
        accmDivests['units'] = accmDivests['units_x'] + accmDivests['units_y'] 
        accmDivests['price'] = (accmDivests['units_x']*accmDivests['price_x'] + accmDivests['units_y']*accmDivests['price_y']) / accmDivests['units']
        accmDivests = accmDivests[['investorID','ticker', 'units', 'price']]
        accmDivests = pd.concat([accmDivests, newDivests])
        accmDivests['divest_period'] = t+1
        accmInvests = accmInvests.merge(accmDivests, on=['investorID', 'ticker'], how='left')
        investReturn = pd.concat([investReturn, accmInvests[accmInvests.units_y.notnull()]])

t= 0
t= 1
t= 2
t= 3
t= 4
t= 5
t= 6
t= 7
t= 8
t= 9
t= 10
t= 11
t= 12
t= 13
t= 14
t= 15
t= 16
t= 17
t= 18
t= 19
t= 20
t= 21
t= 22
t= 23
t= 24


In [157]:
investReturn['holding_period'] = investReturn['divest_period'] -  investReturn['invest_period']
investReturn = investReturn[investReturn['holding_period'] > 0]
investReturn['quarterly_return'] = (investReturn['price_y'] -  investReturn['price_x']) / (investReturn['price_x'] * investReturn['holding_period'])

In [158]:
investReturn[investReturn['invest_period'] == 10].tail()

,investorID,ticker,units_x,price_x,invest_period,units_y,price_y,divest_period,holding_period,quarterly_return
83086,1885,CEQP,568.0,18.921655,10,568.0,36.000000,25.0,15.0,0.060172
83087,2933,KO,33024.0,43.745488,10,33024.0,50.406432,25.0,15.0,0.010151
83088,886,MRK,6129.0,57.707946,10,6129.0,77.280796,25.0,15.0,0.022611
83089,886,BP,3913.0,31.479939,10,3913.0,37.216841,25.0,15.0,0.012149
83090,886,D,1892.0,67.803383,10,1892.0,78.939746,25.0,15.0,0.010950


In [159]:
meanReturn = investReturn[['investorID','holding_period' ,'quarterly_return']]
meanReturn = meanReturn[meanReturn['holding_period']>1][['investorID','quarterly_return']]
meanReturn = meanReturn.groupby('investorID').mean()
meanHoldPeriod = investReturn[['investorID', 'holding_period']]
meanHoldPeriod = meanHoldPeriod.groupby('investorID').mean()
meanInvestStats = meanHoldPeriod.merge(meanReturn, on = ['investorID'], how='left')
meanInvestStats.sort_values(by='quarterly_return', ascending=False).head()

,holding_period,quarterly_return
investorID,,
5061,2.400000,0.619444
5341,3.000000,0.600000
55,7.000000,0.428571
5198,5.221477,0.370556
1905,1.166667,0.335722


In [163]:
investReturn.to_csv("./data/investReturn.csv", header=True, index=False, sep="\t")
meanInvestStats.to_csv("./data/meanInvestStats.csv", header=True, index=True, sep="\t")

In [160]:
investReturn[investReturn['investorID'] == 5061].sort_values(by='quarterly_return', ascending=False).head()

,investorID,ticker,units_x,price_x,invest_period,units_y,price_y,divest_period,holding_period,quarterly_return
362512,5061,IMDZ,6642942.0,1.000000,22,6642942.0,5.0,24.0,2.0,2.000000
335325,5061,CNST,4677939.0,4.000001,22,4677939.0,12.0,25.0,3.0,0.666666
422914,5061,NGM,15694091.0,14.000000,24,15694091.0,14.0,25.0,1.0,0.000000
335323,5061,GRTS,3102934.0,15.000000,22,3102934.0,11.0,25.0,3.0,-0.088889
335324,5061,RCUS,3540850.0,10.000000,22,3540850.0,7.0,25.0,3.0,-0.100000


In [161]:
investorname = df_investors[df_investors['investorID'] == 5061]['investorname'].values[0]
df_SF3[(df_SF3['investorname'] == investorname)&(df_SF3['ticker'] == 'IMDZ')]

,ticker,investorname,securitytype,calendardate,value,units,price
1819961,IMDZ,SVENNILSON PETER,SHR,2019-03-31,38861000,6642942,5.0
3293920,IMDZ,SVENNILSON PETER,SHR,2018-12-31,8636000,6642942,1.0
